In [ ]:
import cv2
import numpy as np
import imutils
import math


def gaussian_blurr(image):
    blurred = cv2.GaussianBlur(image, (35, 35), 0)
    return blurred

def grey_scale(image):
    grey_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return grey_scale

def binarization(image):
    _, threshold = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return threshold

def findContours( threshold ):
    contours, _ = cv2.findContours(threshold.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    return contours

def print_text(img, number_defetcs) : 
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (50, 50) 
    fontScale = 1
    color = (255, 0, 0) 
    thickness = 2
    if count_defects in  [1,2,3,4]:
        cv2.putText(img," Number of fingers : " + str(count_defects), org, font, fontScale, color, thickness, cv2.LINE_AA) 
    else:
        cv2.putText(img, 'Full Hand', org, font, fontScale, color, thickness, cv2.LINE_AA)
    
def extreme_points(cnt):
    extTop = tuple(cnt[cnt[:, :, 1].argmin()][0])
    extTop =  ( extTop[0]  , extTop[1]  ) 
    
    extLeft = tuple(cnt[cnt[:, :, 0].argmin()][0])
    extLeft =  ( extLeft[0]  , extLeft[1]  ) 
    
    extRight = tuple(cnt[cnt[:, :, 0].argmax()][0])
    extRight =  ( extRight[0]  , extRight[1]  ) 

    return extTop, extLeft , extRight

def cosine_rule(defects, img , count):
    number_of_defects = 0
    for i in range(defects.shape[0]):

        S = tuple(count[defects[i,0][0]][0])
        E = tuple(count[defects[i,0][1]][0])
        F = tuple(count[defects[i,0][2]][0])

        a = math.sqrt((E[0] - S[0])**2 + (E[1] - S[1])**2)
        b = math.sqrt((F[0] - S[0])**2 + (F[1] - S[1])**2)
        c = math.sqrt((E[0] - F[0])**2 + (E[1] - F[1])**2)

        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
        if angle <= 90:
            number_of_defects += 1
            cv2.circle(img, F, 1, [0,0,255], -1)
        cv2.line(img,S, E, [100,100,100], 2)
        
    return number_of_defects

#(720, 1280, 3)
    
scale_percent = 60 # percent of original size
width = int(1280 * scale_percent / 100)
height = int(720 * scale_percent / 100)
dim = (width, height)

cap = cv2.VideoCapture(0)

while(True):
    

    if not cap.isOpened():
        break

    ret, img = cap.read()

    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    #cv2.rectangle(img, (300,300), (100,100), (0,255,0),0)
    #crop_img = img[100:300, 100:300]

    crop_img = img
    grey = grey_scale(crop_img)
    
    blurred_image = gaussian_blurr(grey)

    Threshold = binarization(blurred_image)
    
    cv2.imshow('Thresholded', Threshold)

    contours = findContours(Threshold)
    cnt = max(contours, key = lambda x: cv2.contourArea(x))
    
    extTop, extLeft , extRight = extreme_points(cnt)
    
    hull = cv2.convexHull(cnt)
    
    drawing = np.zeros(crop_img.shape,np.uint8)

    cv2.drawContours(drawing, [hull], 0,(0, 0, 255), 0)
    
    hull = cv2.convexHull(cnt, returnPoints=False)

    defects = cv2.convexityDefects(cnt, hull)

    cv2.drawContours(Threshold, contours, -1, (0, 255, 0), 3)
    
    count_defects = cosine_rule(defects , crop_img , cnt)
    print_text(img, count_defects)
    cv2.circle(img, extTop, 20, (255, 255, 0), 4)
    cv2.circle(img, extLeft, 20, (255, 255, 0), 4)
    cv2.circle(img, extRight, 20, (255, 255, 0), 4)
    cv2.imshow('Gesture', img)
    #all_img = np.hstack((drawing, crop_img))
    #cv2.imshow('Contours', all_img)

    k = cv2.waitKey(10)
    if k == 27:
        break
    

In [ ]:
import cv2
import numpy as np
import math


cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    # read image
    ret, img = cap.read()

    # get hand data from the rectangle sub window on the screen
    cv2.rectangle(img, (500,500), (100,100), (0,255,255),0)
    crop_img = img[100:500, 100:500]

    # convert to grayscale
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)

    # applying gaussian blur
    value = (35, 35)
    blurred = cv2.GaussianBlur(grey, value, 0)

    # thresholdin: Otsu's Binarization method
    _, thresh1 = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # show thresholded image
    cv2.imshow('Thresholded', thresh1)

    # check OpenCV version to avoid unpacking error
    (version, _, _) = cv2.__version__.split('.')

    if version == '3':
        image, contours, hierarchy = cv2.findContours(thresh1.copy(), \
               cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    elif version == '4':
        contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \
               cv2.CHAIN_APPROX_NONE)

    # find contour with max area
    cnt = max(contours, key = lambda x: cv2.contourArea(x))

    # create bounding rectangle around the contour (can skip below two lines)
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(crop_img, (x, y), (x+w, y+h), (0, 0, 255), 0)

    # finding convex hull
    hull = cv2.convexHull(cnt)

    # drawing contours
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing, [cnt], 0, (0, 255, 0), 0)
    cv2.drawContours(drawing, [hull], 0,(0, 0, 255), 0)

    # finding convex hull
    hull = cv2.convexHull(cnt, returnPoints=False)

    # finding convexity defects
    defects = cv2.convexityDefects(cnt, hull)
    count_defects = 0
    cv2.drawContours(thresh1, contours, -1, (0, 255, 0), 3)

    # applying Cosine Rule to find angle for all defects (between fingers)
    # with angle > 90 degrees and ignore defects
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]

        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])

        # find length of all sides of triangle
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

        # apply cosine rule here
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57

        # ignore angles > 90 and highlight rest with red dots
        if angle <= 90:
            count_defects += 1
            cv2.circle(crop_img, far, 1, [0,0,255], -1)
        #dist = cv2.pointPolygonTest(cnt,far,True)

        # draw a line from start to end i.e. the convex points (finger tips)
        # (can skip this part)
        cv2.line(crop_img,start, end, [0,255,0], 2)
        #cv2.circle(crop_img,far,5,[0,0,255],-1)

    # define actions required
    if count_defects == 1:
        cv2.putText(img,"This means that we could detect 1 finger", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 2:
        str = "This means that we could detect 2 fingers"
        cv2.putText(img, str, (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    elif count_defects == 3:
        cv2.putText(img,"This means that we could detect 3 fingers", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 4:
        cv2.putText(img,"This means that we could detect 4 fingers", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    else:
        cv2.putText(img,"This means an entire hand", (50, 50),\
                    cv2.FONT_HERSHEY_SIMPLEX, 2, 2)

    # show appropriate images in windows
    cv2.imshow('Gesture', img)
    all_img = np.hstack((drawing, crop_img))
    cv2.imshow('Contours', all_img)

    k = cv2.waitKey(10)
    if k == 27:
        break
